<a href="https://colab.research.google.com/github/Justabhi96/TF2.0/blob/master/16_NLP(Spam_Classification).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q tensorflow==2.0.0-beta1

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, LSTM, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
!wget https://lazyprogrammer.me/course_files/spam.csv

--2019-12-22 11:35:34--  https://lazyprogrammer.me/course_files/spam.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.31.81.48, 104.31.80.48, 2606:4700:30::681f:5030, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.31.81.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 503663 (492K) [text/csv]
Saving to: ‘spam.csv’

spam.csv            100%[===================>] 491.86K  --.-KB/s    in 0.02s   

2019-12-22 11:35:35 (31.7 MB/s) - ‘spam.csv’ saved [503663/503663]



In [0]:
data = pd.read_csv("spam.csv", encoding = "ISO-8859-1")
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [0]:
data = data.iloc[:, :2]
data.columns = ["label", "text"]
data.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [0]:
data["b_label"] = data["label"].map({"ham": 0, "spam": 1})
y = data["b_label"]
print(set(y))

{0, 1}


In [0]:
X_train, X_test, y_train, y_test = train_test_split(data["text"], y, test_size = 0.3)

In [0]:
MAX_VOCAB_SIZE = 20000
tokenizer = Tokenizer(num_words = MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(X_train)
seq_train = tokenizer.texts_to_sequences(X_train)
seq_test = tokenizer.texts_to_sequences(X_test)

In [0]:
print(seq_train[:2])

[[118, 133, 1, 27, 398, 71, 19, 6, 39, 804, 134, 6, 32, 518], [382, 332, 51, 493, 450, 3596, 19, 6, 739, 332, 71, 189, 18, 2, 650, 12, 3597, 189, 450, 28, 740, 18, 2, 650, 12, 1164, 189, 190, 690, 3598, 262]]


In [0]:
seq_train = pad_sequences(seq_train)
print(seq_train.shape)
T = seq_train.shape[1]

(3900, 189)


In [0]:
seq_test = pad_sequences(seq_test, maxlen = T)
seq_test.shape

(1672, 189)

In [0]:
V = len(tokenizer.word_index)
V

7322

In [0]:
D = 20 # features
M = 15 # hidden

i = Input(shape = (T,))
x = Embedding(V+1, D)(i)
x = LSTM(M, return_sequences = True)(x)
x = GlobalMaxPooling1D()(x)
x = Dense(1, activation = "sigmoid")(x)

model = Model(i, x)
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
r = model.fit(seq_train, y_train, validation_data = (seq_test, y_test), epochs = 10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 3900 samples, validate on 1672 samples
Epoch 1/10
3900/3900 [==============================] - 13s 3ms/sample - loss: 0.4699 - accuracy: 0.8528 - val_loss: 0.3731 - val_accuracy: 0.8774
Epoch 2/10
3900/3900 [==============================] - 12s 3ms/sample - loss: 0.4035 - accuracy: 0.8610 - val_loss: 0.3733 - val_accuracy: 0.8774
Epoch 3/10
3900/3900 [==============================] - 12s 3ms/sample - loss: 0.4031 - accuracy: 0.8610 - val_loss: 0.3740 - val_accuracy: 0.8774
Epoch 4/10
3900/3900 [==============================] - 12s 3ms/sample - loss: 0.4021 - accuracy: 0.8610 - val_loss: 0.3699 - val_accuracy: 0.8774
Epoch 5/10
3900/3900 [==============================] - 12s 3ms/sample - loss: 0.3859 - accuracy: 0.8610 - val_loss: 0.3405 - val_accuracy: 0.8774
Epoch 6/10
3900/3900 [==============================] - 12s 3ms/sample - loss: 0.2785 - accuracy: 0.8721 - val_loss: 0.2018